In [1]:
#@title #setup { display-mode: "form"}
#import libaries
#==============
#NumPy libaries
import numpy as np
from numpy import heaviside
#SciPy libaries
from scipy.integrate import quad, quad_vec, dblquad
from scipy.integrate import quad
#SymPy libaries
import sympy as sp
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
from sympy import sin, cos, exp, pi, symbols
from sympy.vector import cross, CoordSys3D, ParametricRegion, ImplicitRegion, vector_integrate
from sympy import init_printing
from sympy.printing import latex
#plotting libaries
!pip install ipympl > /dev/null #install new library
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()
#==============
#Print in LaTex setup:
#==============
from IPython.display import display, Latex
#%precision %.4g

def safe_sign(x):
  x = np.where(np.isnan(x), 0, x)
  x = np.where(x < 1, 0, x)
  return x

In [2]:
#@title #physics constants { display-mode: "form"}
me = 9.11e-31
display(Latex(rf"m_e = {me}"))
hbar = 1.054e-34
display(Latex(rf"\hbar = {hbar}"))
EnergyOfElectron = 1.6e-19
display(Latex(rf"1 eV = {EnergyOfElectron}J"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [3]:
#@title #question parameters { display-mode: "form"}
x = 0.6
display(Latex(rf"x = {x}"))
Eg1 = 1.673
Eg2 = 1.424
deltaEc = x*(Eg1-Eg2)
display(Latex(rf"\Delta E_c = x \cdot \left( E_{{g1}} - E_{{g2}} \right) = {deltaEc:.4f} [eV]"))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [4]:
#@title #physics equations { display-mode: "form"}
V = deltaEc * EnergyOfElectron
N = 2 * me * V
D = hbar ** 2
k0 = (N / D) ** 0.5

N = D
D = 2 * me * EnergyOfElectron
En = (N / D) * 1e21
print(En)

38.10771130625685


In [5]:
#@title #find intersection function { display-mode: "form"}

def intersection(arr1, arr2):
    intersect=[]
    index = 0
    while index < len(arr1):
      if index < 10 or index == len(arr1) - 10:
          index += 1
          continue
      for i in range(10):
          if arr2[index] > arr1[index - i] and arr2[index] < arr1[index + i] and not np.isnan(arr2[index]):
              #print(index)
              intersect.append(arr2[index])
              #print(arr2[index])
              index = index + 100
              break
      index += 1
    return intersect




In [6]:
#@title #plot modes with given length: plot_modes(L) { display-mode: "form"}

def plot_modes(L):
    k = np.linspace(1e8, 3e9, 2000)

    evenModes = []
    oddModes = []
    for index, ki in enumerate(k):
        signK = np.tan(ki * L / 2)
        if signK > 0:
            eM = k0 * np.abs(np.cos(ki * L / 2))
            oddModes.append(np.nan)
            evenModes.append(eM)
        elif signK < 0:
            oM = k0 * np.abs(np.sin(ki * L / 2))
            oddModes.append(oM)
            evenModes.append(np.nan)
        else:
            oddModes.append(np.nan)
            evenModes.append(np.nan)
    intersect = np.concatenate([intersection(k, evenModes), intersection(k, oddModes)])
    intersect.sort()
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Modes of energy, and intersect line', 'Discrete energy levels in finite potential well'])
    #graph to find the intersect points
    fig.add_trace(go.Scatter(
        x = k,
        y = evenModes, name = "Even Mode"),row=1, col=1)
    fig.add_trace(go.Scatter(
        x = k,
        y=oddModes, name = "Odd Mode"),row=1, col=1)
    fig.add_trace(go.Scatter(
        x = k,
        y = k,
        name = "k value"),row=1, col=1)
    fig.add_trace(go.Scatter(x=intersect, y=intersect,
                    mode='markers',
                    name='intersection points',
                    marker=dict(
                            color='LightSkyBlue',
                            size=20,
                            line=dict(
                                color='MediumPurple',
                                width=2
                                      ))),row=1, col=1)
    fig.update_layout(
        yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'),
        xaxis = dict(
        showexponent = 'all',
        exponentformat = 'e')
    )
    fig.update_xaxes(title_text='K', row=1, col=1)
    fig.update_yaxes(title_text='K', row=1, col=1)
    #graph to plot the discrete energies levels inside the hole
    EN = (En*intersect**2)*1e-18
    L = L*1e9
    mid = np.linspace(-L/2, L/2, 5)
    left = np.linspace(-L, -L/2, 5)
    right = np.linspace(L/2, L, 5)
    combined_length = np.concatenate((left, mid, right))
    #plot energy level in well
    holeY=[]
    for i in left: holeY.append(deltaEc*1e3)
    for i in mid: holeY.append(0)
    for i in right: holeY.append(deltaEc*1e3)
    fig.add_trace(go.Scatter(
        x = combined_length,
        y = holeY, name = "Finite Potential well", mode='lines', fill='tozeroy'),row=1, col=2)
    for Discrete_energy in EN:
      fig.add_trace(go.Scatter(
          x = mid,
          y = Discrete_energy*np.ones(len(mid)),
          mode = 'lines',
          name = rf"Discrete energy {Discrete_energy:.2f}"
      ), row = 1, col = 2)
    fig.update_xaxes(title_text='Distance [nm]', row=1, col=2)
    fig.update_yaxes(title_text='Energy [eV]', row=1, col=2)
    fig.show()
    display(Latex(rf"L = {L:.2f} [nm]"))
    display(Latex(rf"E_{{i}}=\frac{{\hbar ^2}}{{2\cdot m_e \cdot {EnergyOfElectron}}}\cdot {{k_i^2}} = {En:.2f}\cdot {{k_i^2}} [meV]"))
    latex_code = rf"E_n =  \left( {',⠀'.join(map(lambda x: f'{x:.2f}', EN))} \right)[meV]"
    display(Latex(latex_code))
    return EN


In [ ]:
en1 = plot_modes(10e-9)
en2 = plot_modes(20e-9)

In [8]:
#@title #plot for infinite potential well vs  finite potential well given discrete energy levels and length: plot_infinite_potential_well(L, En) { display-mode: "form"}
def plot_infinite_potential_well(En, L):
    en = [0]
    en.extend(En)
    En = en
    x_values = list(range(0, len(En)))
    #print(x_values)
    #print(En)
    infinite_potential_well_E= (np.pi**2*hbar**2)/(2*me*EnergyOfElectron*L**2)
    infLine=[]
    for n in x_values:
      infLine.append(infinite_potential_well_E*n**2 *1e3) #13e to get results in [m*eV]
    #print(infLine)
    y_diff_values = [y2 - y1 for y1, y2 in zip(En, infLine)]
    # Create the traces
    trace1 = go.Scatter(x=x_values, y=infLine, mode='lines+markers', name='infinite potential well',
                        line=dict(color='darkblue', width=1,
                              dash='dash'))
    trace2 = go.Scatter(x=x_values, y=En, mode='markers', name='finite potential well')
    # Create a trace for the differences
    highs = infLine
    closes = infLine
    opens = En
    lows = En
    # Calculate the differences between high and low
    delta_values = [high - low for high, low in zip(highs, lows)]
    # add bar of delta between points
    trace_diff =go.Candlestick(x=x_values,
                       open=opens,
                       high=highs,
                       low=lows,
                       close=closes,
                        name='Delta Energy',
                        increasing_fillcolor='cyan',
                               decreasing_line_color='cyan',
                               increasing_line_width=0,
                               decreasing_line_width=0)

    # Add trace for delta values inside each bar
    trace_delta = go.Scatter(x=x_values, y=[max(high, low) for high, low in zip(highs, lows)],
                         mode='text',
                         text=[f'Delta: {delta:.2e}' for delta in delta_values],
                         textposition='top center',
                         showlegend=False)

    # Create the layout
    l = L*1e9
    layout = go.Layout(title=rf"Difference in energy values between infinite potential well and finite potential well, length = {l} [nm]",
                      xaxis=dict(title='n value'),
                      yaxis=dict(title='Energy [eV]'),
                       xaxis_rangeslider_visible=False)

    # Create the figure
    #fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig = go.Figure(data=[trace1, trace2, trace_diff, trace_delta], layout=layout)
    # Show the figure
    fig.show()

In [ ]:
plot_infinite_potential_well(en1, 10e-9)
plot_infinite_potential_well(en2, 20e-9)

In [18]:
#@title #plot for schrodinger equation for finite potential well { display-mode: "form"}

import numpy as np
import plotly.graph_objs as go

me = 9.11e-31  # kg
hbar = 1.054e-34  # J*s
en = np.array([3.31809807, 12.33255392, 28.00366818,
               49.21907726, 76.3887979, 108.25413403,
               143.05688807]) * 1e-3 * 1.60218e-19  # [J]
V = 149 * 1e-3 * 1.60218e-19  # [J]
L = 10e-9  # [m]

def Wave_function(E, V, L, A):
    k = np.sqrt(2 * me * E) / hbar
    rho = np.sqrt(2 * me * (V - E)) / hbar

    B = -1j * (rho + 1j * k) * A * np.exp((-rho + 1j * k) * L / 2) / (2 * k)
    C = 1j * (rho - 1j * k) * A * np.exp(-(rho + 1j * k) * L / 2) / (2 * k)
    D = (rho**2 + k**2) * A * np.sin(k * L) / (2 * k * rho)

    X = np.concatenate((
        np.linspace(-L, -L / 2, 100),
        np.linspace(-L / 2, L / 2, 1000),
        np.linspace(L / 2, L, 100)))

    Phi = []

    for x in X:
        if x < -L / 2 or x == -L/2:
            value = A * np.exp(rho * x)
        elif -L / 2 < x < L / 2:
            value = B * np.exp(1j * k * x) + C * np.exp(-1j * k * x)
        else:
            value = D * np.exp(-rho * x)

        Phi.append(np.real(value))
        #Phi.append(np.abs(value))  # Changed this line to get the absolute value

    return Phi, X

def plot_wave_function(Earray, V, L, A):
    data = []
    for i, energy in enumerate(Earray):
        Phi, X = Wave_function(energy, V, L, A)
        trace = go.Scatter(x=X, y=Phi, mode='lines', name=rf"{en[i]* 1e3 * (1.60218e-19)**(-1)} meV")
        data.append(trace)

    # Add the finite potential well
    trace_well = go.Scatter(x=[-L/2, -L/2, L/2, L/2],
                            y=[0, max(Phi), max(Phi), 0],
                            mode='lines',
                            name="Finite Potential Well",
                            line=dict(color='black', width=2, dash='dash'))
    data.append(trace_well)

    fig = go.Figure(data=data)
    fig.update_layout(
        title="Wave Function for Finite Potential Well",
        xaxis_title="Position (m)",
        yaxis_title="Wave Function Amplitude",
        showlegend=True
    )
    fig.show()

# Example usage:
plot_wave_function(en, V, L, A=1)  # You can adjust the scalar A as needed
